**Title : Product attribute Extraction**

Group Members:-

Manas pandey :- 203100065

Mudit sand :- 203100068

Supriyo Roy :- 193109013


Importing Libraries

In [ ]:
%tensorflow_version 2.x
!pip install tensorflow
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.initializers import Constant


Loading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/train.csv')

Visualizinig the dataset

In [ ]:
dataset.head()

,Unnamed: 0,title,brand,present,root_cat
0,0,Galt Shape Puzzle Emergency Vehicles Baby/Todd...,Galt,True,2984
1,1,BRIO Magnetic Building Wooden Blocks Toddler A...,Brio,True,2984
2,2,NEW GRAFIX 6PCS METALLIC FABRIC PAINT PEN MARK...,GRAFIX,True,550
3,3,NEW GRAFIX 6PCS METALLIC FABRIC PAINT PEN MARK...,GRAFIX,True,550
4,4,Coca Cola Coke Beach Girl Large 24 x 36 Wall P...,Coca Cola,True,550


Taking the important features of the dataset for further processing

In [ ]:
dataset= dataset[['title','brand']]

We will do the tokenization of the words persent in the dataset. Tokenizer will split the data which will then be used for mapping as a numeric vector by 
the help of glove.

In [ ]:
tokenlist = []
tokenlist.append(dataset['title'])
tokenlist.append(dataset['brand'])
tokenlist = np.ravel(tokenlist)
print(tokenlist)

['Galt Shape Puzzle Emergency Vehicles Baby/Toddler/Child Jigsaw Shapes BNIB'
 'BRIO Magnetic Building Wooden Blocks Toddler Activity Toy Ages 1+ NEW'
 'NEW GRAFIX 6PCS METALLIC FABRIC PAINT PEN MARKER T SHIRT CLOTHES ART 6 COLOUR'
 ... 'FOLKESPEARE' 'FOLKESPEARE' 'TOOTAL']


In [ ]:
featurestoCare = 20000
tokens = Tokenizer(num_words=featurestoCare)
tokens.fit_on_texts(tokenlist)
x = tokens.texts_to_sequences(dataset['title'].values)
x = pad_sequences(x)


In [ ]:
y = dataset['brand'].values
y = tokens.texts_to_sequences(y)
y = pad_sequences(y)

Train-Test Data Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1,random_state=44)

Word Embedding

In [ ]:
embeddings_index = {}

# Glove Embedding Link: https://nlp.stanford.edu/projects/glove/
f = open('/content/drive/MyDrive/glove.6B.300d.txt') #Glove Embedding File 
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
word_index = tokens.word_index
print('Found %s unique tokens.' % len(word_index))

Found 33726 unique tokens.


In [ ]:
embedding_dim = 300
num_words = min(featurestoCare, len(word_index)) + 1
# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))
# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    elif embedding_vector is None:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)



In [ ]:
T = X_train.shape[1]

Bi-LSTM Model

In [ ]:
model = Sequential()
# Embedding Layer 
model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=T,
                    trainable=True))

# Bi_LSTM Layer
model.add(Bidirectional(LSTM(100,return_sequences=True, go_backwards= True)))
model.add((LSTM(100, go_backwards= True)))

#Dense Layer
model.add(Dense(5, activation = 'relu'))

#Optimizer
Nadam = tf.keras.optimizers.Nadam(learning_rate=1.5)

#Model Compilation
model.compile(loss = 'hinge', optimizer=Nadam,metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 21, 300)           6000300   
_________________________________________________________________
bidirectional (Bidirectional (None, 21, 200)           320800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense (Dense)                (None, 5)                 505       
Total params: 6,442,005
Trainable params: 6,442,005
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Trainining Data
print(X_train)

[[   0    0    0 ... 1284 1199  213]
 [   0    0    0 ...    3   13   15]
 [   0    0    0 ...  988 6484 3694]
 ...
 [   0    0    0 ... 3305  783  153]
 [   0    0    0 ... 1207  120   95]
 [   0    0    0 ...  295 1474  452]]


In [ ]:
y_train.shape

(27000, 5)

Model Fitting

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Epoch 1/3
844/844 [==============================] - 122s 139ms/step - loss: 0.8002 - accuracy: 0.8953 - val_loss: 0.7995 - val_accuracy: 0.9073
Epoch 2/3
844/844 [==============================] - 114s 135ms/step - loss: 0.7997 - accuracy: 0.9007 - val_loss: 0.7995 - val_accuracy: 0.9073
Epoch 3/3
844/844 [==============================] - 114s 135ms/step - loss: 0.7997 - accuracy: 0.9001 - val_loss: 0.7995 - val_accuracy: 0.9073


Prediction in Vector

In [ ]:
ypre = model.predict(X_test)

In [ ]:
print(ypre)

[[  0.          0.7851577   0.          0.        143.4151   ]
 [  0.          0.7431967   0.          0.        144.58968  ]
 [  0.          0.7772007   0.          0.        143.95114  ]
 ...
 [  0.          0.7855067   0.          0.        143.77292  ]
 [  0.          0.696684    0.          0.        139.81114  ]
 [  0.          0.4439757   0.          0.        149.30191  ]]


In [ ]:
len = ypre.shape[0]

Converting the prediction from float to integer

In [ ]:
i=0
while i< len:
  ypre[i][4] = round(ypre[i][4])
  ypre[i][3] = round(ypre[i][3])
  ypre[i][2] = round(ypre[i][2])
  ypre[i][1] = round(ypre[i][1])
  ypre[i][0] = round(ypre[i][0])
  i = i+1

Vector to Word

In [ ]:
predicted = tokens.sequences_to_texts(ypre)

Prediction in Words

In [ ]:
print(predicted)

['mens hdd', 'mens small', 'mens 4gb', 'mens hdd', '15', 'mens small', 'mens sas', 'mens 4gb', 'mens 4gb', 'mens hdd', 'mens 4gb', 'mens motorola', 'mens 4gb', 'mens 4gb', 'mens 4gb', 'hard', 'mens 4gb', 'hard', 'hard', 'mens motorola', 'mens shoes', 'mens sas', 'hard', 'mens motorola', 'mens sas', 'mens 4gb', 'mens small', 'mens pair', 'hard', 'mens hdd', 'mens 4gb', 'mens hdd', 'mens motorola', 'hard', 'mens 4gb', 'mens motorola', 'hard', 'mens sas', 'mens 4gb', 'mens hdd', 'hard', 'mens 4gb', 'hard', 'mens motorola', 'mens pair', 'mens 4gb', 'hard', 'mens hdd', 'sas', 'mens 4gb', 'mens 4gb', 'mens 4gb', 'mens hdd', 'hard', 'mens sas', 'mens hdd', 'mens hdd', 'mens small', 'mens hdd', 'mens sas', 'mens hdd', 'mens 4gb', 'mens motorola', 'hdd', 'mens hdd', 'hard', 'mens 4gb', 'mens hdd', 'hard', 'mens 4gb', 'mens 4gb', 'hard', 'mens 4gb', 'mens hdd', 'hard', 'mens 4gb', 'mens hdd', 'hard', 'hard', 'mens hdd', 'mens 4gb', 'mens hdd', 'mens hdd', 'hard', 'mens 4gb', 'mens hdd', 'mens 4g